# Variant Calling

In [2]:
import pandas as pd
import numpy as np

datapath = 'data/'

def read_vcf(filename):
    df = pd.read_csv(datapath + filename, sep='\t')
    
    # filter to separate indels from snp
    snp_only = df.apply(lambda row: len(row.REF)==1 and len(row.ALT)==1, axis=1)
    snp = df[snp_only]
    indels = df[~snp_only]
    
    print len(df), len(snp), len(indels)
    
    return snp, indels

### 1. read NIST.vcf

In [3]:
nist2_snp, nist2_indels = read_vcf('na12878-nist.chr2')
nist3_snp, nist3_indels = read_vcf('na12878-nist.chr3')
nist3_snp.head()

373001 303275 69726
306338 249746 56592


,CHROM,POS,REF,ALT,DP,ADR,ADA,GQ,LABEL
0,3,60596,C,A,40,0,40,99,FAIL
1,3,61044,T,C,34,0,34,99,FAIL
2,3,61113,A,T,30,0,30,90,FAIL
3,3,61495,A,G,49,24,25,99,FAIL
4,3,61762,T,A,41,17,24,99,FAIL


### 2. read garvan.vcf

In [4]:
garv2_snp, garv2_indels = read_vcf('na12878-garvan.chr2')
garv3_snp, garv3_indels = read_vcf('na12878-garvan.chr3')
garv3_snp.head()

373001 303275 69726
306338 249746 56592


,CHROM,POS,REF,ALT,DP,ADR,ADA,GQ,LABEL
0,3,60596,C,A,40,0,40,99,PASS
1,3,61044,T,C,34,0,34,99,PASS
2,3,61113,A,T,30,0,30,90,PASS
3,3,61495,A,G,49,24,25,99,PASS
4,3,61762,T,A,41,17,24,99,PASS


### 3. NIST_train = (garvan if in NIST.vcf - NIST_chr3)

The NIST dataset doesn't contain the AD feature, so we need to get that column from the Garvan dataset, using an inner join.

In [5]:
#ng_inner = garv_train.merge(nist_train, how='inner', copy=True, indicator=True, 
#                             fields=['CHROM', 'POS', 'REF', 'ALT'])

nist_train = nist3_snp
nist_test = nist3_snp

### 4. garvan_train = split(garvan.vcf - garban_chr3)

In [6]:
garv_train = garv2_snp
garv_test = garv3_snp

### 5. est_neg  = (garvan_train - NIST_train),  (change to fail)

In [7]:
"""joined = garv_train.merge(nist_train, how='left', on=['CHROM', 'POS', 'REF', 'ALT'], 
                             copy=True, indicator=True)
grouped = joined.groupby('_merge')
for g in grouped.groups:
    print g
joined.head()"""

"joined = garv_train.merge(nist_train, how='left', on=['CHROM', 'POS', 'REF', 'ALT'], \n                             copy=True, indicator=True)\ngrouped = joined.groupby('_merge')\nfor g in grouped.groups:\n    print g\njoined.head()"

In [8]:
"""est_neg = joined[['CHROM_x', 'POS', 'REF', 'ALT', 'DP_x', 'ADR_x', 'ADA_x', 'GQ_x', 'LABEL_x']][joined._merge!='left_only']
est_neg.columns = ['CHROM', 'POS', 'REF', 'ALT', 'DP', 'ADR', 'ADA', 'GQ', 'LABEL']
est_neg['LABEL'] = 'FAIL'
est_neg.head()
joined = None"""

"est_neg = joined[['CHROM_x', 'POS', 'REF', 'ALT', 'DP_x', 'ADR_x', 'ADA_x', 'GQ_x', 'LABEL_x']][joined._merge!='left_only']\nest_neg.columns = ['CHROM', 'POS', 'REF', 'ALT', 'DP', 'ADR', 'ADA', 'GQ', 'LABEL']\nest_neg['LABEL'] = 'FAIL'\nest_neg.head()\njoined = None"

### 6. train = nist_train + est_neg

In [9]:
"""train = pd.concat([nist_train, est_neg])
est_neg = None"""
train = nist_train

### 7. classifier = model.fit(train)

In [10]:
train_y = np.array(train['LABEL']=='PASS')
print len(train_y), sum(train_y)

249746 197419


In [11]:
def extract_features(obs, cols):
    bases = ['A', 'C', 'G', 'T']
    X = np.ndarray(shape=(len(obs), len(cols) + 2*(len(['A', 'C', 'G', 'T'])-1)))
    i = 0
    for ix in obs.index:
        row = obs.ix[ix]
        rowd = {}
        for c in cols:
            if c in ['REF', 'ALT']:  
                for n in bases:
                    rowd[c + '_' + n] = 0.0
                rowd[c+'_'+row[c]] = 1.0
            else:
                if ' ' in c: print c
                elif '\t' in c: print c
                elif ',' in c: print c
                rowd[c] = row[c] 
                
        X[i,:] = rowd.values()
        i += 1
        
    feature_names = []
    for c in cols:
        if c in ['REF', 'ALT']:  
            for n in bases:
                feature_names.append(c + '_' + n)
        else:
            feature_names.append(c) 
    
    return X, feature_names 

In [12]:
columns = ['REF', 'ALT', 'DP', 'ADR', 'ADA', 'GQ']
train_X, feature_names = extract_features(train, columns)
train_X[0:1,:]

array([[  1.,   0.,   0.,  99.,  40.,   0.,  40.,   0.,   1.,   0.,   0.,
          0.]])

In [13]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state=1)
classifier.fit_transform(train_X, train_y)
print feature_names
print classifier.coef_

['REF_A', 'REF_C', 'REF_G', 'REF_T', 'ALT_A', 'ALT_C', 'ALT_G', 'ALT_T', 'DP', 'ADR', 'ADA', 'GQ']
[[-0.35775884 -0.46846063 -0.35028482  0.02818711  0.60571052 -0.4466496
  -0.56878602 -0.45733557 -0.45713656  0.58770187 -0.37373917 -0.35631454]]


/home/adrian/anaconda2/lib/python2.7/site-packages/sklearn/utils/__init__.py:93: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)


### 8. Eval

####    8.1   -  test = nist3 + changed_to_fail(garvan3 - nist3) 

In [14]:
columns = ['REF', 'ALT', 'DP', 'ADR', 'ADA', 'GQ']
test_X, feature_names = extract_features(nist_test, columns)
test_X[0:1,:]

array([[  1.,   0.,   0.,  99.,  40.,   0.,  40.,   0.,   1.,   0.,   0.,
          0.]])

In [15]:
test_y = np.array(nist_test['LABEL']=='PASS')
print len(test_y), sum(test_y), len(test_y) - sum(test_y), "Majority Class Score: ", sum(test_y)/float(len(test_y))

249746 197419 52327 Majority Class Score:  0.790479126793


####    8.2   -  pred_y = model.predict(test.X)

In [16]:
pred_y = classifier.predict(test_X)

####    8.3 - Accuracy Score 
pred_y vs test_y

In [22]:
classifier.score(test_X, test_y)

0.80226710337703111

 ### Confusion Matrix

In [34]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(test_y, pred_y)

label_dict = {0: '0-FAIL', 1:'1-PASS'}
print '     ', label_dict.values()
for i in range(len(cm[:,0])):
    print label_dict [i], cm[i,:]

      ['0-FAIL', '1-PASS']
0-FAIL [ 4304 48023]
1-PASS [  1360 196059]


 $C_{i, j}$ is equal to the number of observations known to be in group i but predicted to be in group j.

In [37]:
tn = cm[0,0]
fp = cm[0,1]
fn = cm[1,0]
tp = cm[1,1]
acu = (tp+tn)/float(tp+fp+tn+fn)
pre = (tp)/float(tp+fp)
rec = (tp)/float(tp+fn)
print "Accuracy   = %.4f" % acu
print "Precission (specificity) = %.4f" % pre
print "Recall     (sensitivity) = %.4f" % rec

Accuracy   = 0.8023
Precission (specificity) = 0.8033
Recall     (sensitivity) = 0.9931


### F1 score (also F-score or F-measure) 
The traditional F-measure or balanced F-score (F1 score) is the harmonic mean of precision and recall:

$$F_1 = 2 \cdot \frac{\mathrm{precision} \cdot \mathrm{recall}}{\mathrm{precision} + \mathrm{recall}}$$

In [36]:
F_measure = 2 * pre * rec / (pre + rec)
print "F_measure = %.4f" % F_measure

F_measure = 0.8881
